In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/ExpW/data/image"

origin.7z.001  origin.7z.003  origin.7z.005  origin.7z.007
origin.7z.002  origin.7z.004  origin.7z.006  origin.7z.008


In [ ]:
!ls
!7za -y x "/content/drive/MyDrive/Colab Notebooks/ExpW/data/image/origin.7z.*"

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!cp -r ./origin/ "/content/drive/MyDrive/Colab Notebooks"

In [ ]:
import numpy as np
import os
import glob
import pandas as pd
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [ ]:
labels_file = "/content/drive/MyDrive/Colab Notebooks/ExpW/data/label/label.lst"
df = pd.read_csv(labels_file, header=None, delimiter=" ", names=['image_name', 'face_id_in_image', 'face_box_top', 'face_box_left', 'face_box_right', 'face_box_bottom', 'face_box_cofidence', 'expression_label'])
df

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
...,...,...,...,...,...,...,...,...
91788,surprised_expression_546.jpg,0,70,70,351,351,37.7117,5
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5
91791,ecstatic_asian_31.jpg,0,60,136,184,108,39.9223,3


In [ ]:
for ind in df.index:
  df_iterate = df['image_name'][ind], df['face_box_cofidence'][ind]

print(df_iterate)

('surprised_expression_394.jpg', 77.7758)


In [ ]:
df_sel = df[df.face_box_cofidence > 65]
df_sel

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
5,angry_actor_137.jpg,0,93,468,842,467,88.9519,0
9,angry_actor_150.jpg,0,56,263,376,169,81.8792,0
11,angry_actor_16.jpg,0,158,249,521,430,66.4535,0
...,...,...,...,...,...,...,...,...
91783,afraid_woman_204.jpg,0,20,13,93,100,74.5950,6
91787,expressionless_husband_673.jpg,0,48,194,388,242,82.2975,4
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5


In [ ]:
for i, row in df_sel.iterrows():
  image_name = row['image_name']
  x1 = row['face_box_left']
  x2 = row['face_box_right']
  y1 = row['face_box_top']
  y2 = row['face_box_bottom']
  a = [x1,x2, y1,y2]

print(a)

[38, 152, 47, 161]


In [ ]:
import os
import cv2
from tqdm import tqdm_gui

X = []
Y = []

image_directory = '/content/drive/MyDrive/Colab Notebooks/origin'

for i, row in df_sel.sample(2000).iterrows():
    image_name = row['image_name']
    x1 = row['face_box_left']
    x2 = row['face_box_right']
    y1 = row['face_box_top']
    y2 = row['face_box_bottom']
    label = row['expression_label']

    comb_path = os.path.join(image_directory, image_name)
    cropped_image = cv2.imread(comb_path)

    if cropped_image is not None:
        face = cropped_image[x1:x2, y1:y2]
        print(f"Face shape: {face.shape}, x1={x1}, x2={x2}, y1={y1}, y2={y2}")
    else:
        continue

    if face is not None and not face.size == 0:
        face_resized = cv2.resize(face, (64, 64))
    else:
        continue

    X.append(face_resized)
    Y.append(label)

Face shape: (67, 67, 3), x1=67, x2=134, y1=16, y2=83
Face shape: (133, 231, 3), x1=250, x2=481, y1=19, y2=250
Face shape: (48, 48, 3), x1=236, x2=284, y1=272, y2=320
Face shape: (161, 161, 3), x1=67, x2=228, y1=134, y2=295
Face shape: (67, 67, 3), x1=90, x2=157, y1=45, y2=112
Face shape: (61, 80, 3), x1=1272, x2=1352, y1=363, y2=443
Face shape: (323, 323, 3), x1=242, x2=565, y1=296, y2=619
Face shape: (94, 94, 3), x1=173, x2=267, y1=39, y2=133
Face shape: (47, 47, 3), x1=59, x2=106, y1=39, y2=86
Face shape: (67, 67, 3), x1=118, x2=185, y1=28, y2=95
Face shape: (67, 67, 3), x1=332, x2=399, y1=168, y2=235
Face shape: (0, 48, 3), x1=381, x2=429, y1=112, y2=160
Face shape: (48, 48, 3), x1=152, x2=200, y1=8, y2=56
Face shape: (81, 81, 3), x1=216, x2=297, y1=87, y2=168
Face shape: (273, 273, 3), x1=68, x2=341, y1=68, y2=341
Face shape: (287, 387, 3), x1=129, x2=516, y1=0, y2=387
Face shape: (80, 80, 3), x1=202, x2=282, y1=47, y2=127
Face shape: (462, 462, 3), x1=500, x2=962, y1=154, y2=616
F

In [ ]:
print(len(X))
print(len(Y))

1242
1242


In [ ]:
X = np.array(X)
Y = np.array(Y)
print(len(X))

1242


In [ ]:
if isinstance(X, np.ndarray) and isinstance(Y, np.ndarray):
    print("Images are converted to NumPy arrays.")
else:
    print("Images are not converted to NumPy arrays.")

Images are converted to NumPy arrays.


In [ ]:
X_normalized = X.astype('float32') / 255.0

print("Shape of X_normalized:", X_normalized.shape)

Shape of X_normalized: (1242, 64, 64, 3)


In [ ]:
if np.min(X_normalized) >= 0.0 and np.max(X_normalized) <= 1.0:
    print("Images are properly normalized.")
else:
    print("Images are not properly normalized. Pixel value range: [", np.min(X_normalized), ",", np.max(X_normalized), "]")

Images are properly normalized.


In [ ]:
X_new = X.reshape(-1, 64*64*3)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

sampler = RandomOverSampler()

if len(X_new.shape) > 2:
    X_new = X_new.reshape(-1, 64*64*3)

XSMOTE, YSMOTE = sampler.fit_resample(X_new, Y)

if len(X_new.shape) > 2:
    XSMOTE = XSMOTE.reshape(-1, 64, 64, 3)

In [ ]:
pd.Series(YSMOTE).value_counts()

3    622
1    622
6    622
5    622
2    622
4    622
0    622
dtype: int64

In [ ]:
X = XSMOTE.reshape(-1, 64, 64, 3)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded.reshape(-1, 1))

print("Shape of onehot_encoded:", onehot_encoded.shape)

Shape of onehot_encoded: (1242, 7)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
print("First few rows of onehot_encoded:")
print(onehot_encoded[:5])

First few rows of onehot_encoded:
[[0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True
)

datagen.fit(X)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X_normalized, onehot_encoded, test_size=0.2, random_state=42)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(onehot_encoded.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

batch_size = 32
epochs = 10
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

Epoch 1/10
28/28 [==============================] - 10s 338ms/step - loss: 1.6160 - accuracy: 0.4401 - val_loss: 1.4070 - val_accuracy: 0.5600
Epoch 2/10
28/28 [==============================] - 7s 239ms/step - loss: 1.4766 - accuracy: 0.4927 - val_loss: 1.3336 - val_accuracy: 0.5600
Epoch 3/10
28/28 [==============================] - 5s 190ms/step - loss: 1.4228 - accuracy: 0.4793 - val_loss: 1.3155 - val_accuracy: 0.5600
Epoch 4/10
28/28 [==============================] - 7s 268ms/step - loss: 1.4085 - accuracy: 0.4882 - val_loss: 1.3825 - val_accuracy: 0.5300
Epoch 5/10
28/28 [==============================] - 5s 179ms/step - loss: 1.4017 - accuracy: 0.4782 - val_loss: 1.3745 - val_accuracy: 0.5600
Epoch 6/10
28/28 [==============================] - 6s 232ms/step - loss: 1.3518 - accuracy: 0.4826 - val_loss: 1.3111 - val_accuracy: 0.5700
Epoch 7/10
28/28 [==============================] - 6s 201ms/step - loss: 1.3082 - accuracy: 0.5028 - val_loss: 1.3251 - val_accuracy: 0.5400
Epoch